# Steady State Solve

Example is based on Julian's Laplace example.

In [1]:
import petsc4py.PETSc as PETSc
import underworld3 as uw

In [2]:
user = {"y0": -0.6,
        "y1": 1.3,
        "k": 0.5,
        "h": 10,
        "T0": 4,
        "T1": 8,
        "simplex": False}

## Create Mesh

In [3]:
mesh = uw.FeMesh(elementRes=[6,6], minCoords=(-1.0, -1.0), maxCoords=(1.0, 1.0), simplex=True)
mesh.save("mesh.h5")

In [4]:
part = mesh.plex.getPartitioner()
part.setFromOptions()
mesh.plex.distribute()

In [5]:
mesh.plex.localizeCoordinates()
mesh.plex.setFromOptions()

## Setup Discretisation

In [6]:
# Get Communicator from DM
comm = mesh.plex.comm
# Create Default FE
fe_temp = PETSc.FE().createDefault(dim=mesh.dim,
                                   nc=1,
                                   isSimplex=mesh.isSimplex,
                                   prefix="temperature_",
                                   qorder=-1, comm=comm)
mesh.plex.setField(0, fe_temp)
mesh.plex.createDS()
prob = mesh.plex.getDS()

### Setup Problems

In [7]:
from underworld3.poisson import PoissonSetup

In [8]:
PoissonSetup(mesh.plex, prob, user)